In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install torchinfo

In [3]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import shutil
import torch
from torch import nn
from torchinfo import summary
import cv2
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import albumentations as A
import cv2
import zipfile
from torchvision import transforms, datasets
from PIL import Image
import matplotlib.pyplot as plt
from time import perf_counter as timer
import math
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from typing import Callable, Tuple, Optional, Dict, Any
import matplotlib.pyplot as plt


import numpy as np
import torch
import torch.nn as nn
from typing import Dict, Any, List, Tuple, Optional
import random
from sklearn.model_selection import ParameterSampler
from scipy.stats import uniform, loguniform, randint
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import json

In [4]:
# Your existing data loading code:
train_data_path = "/content/drive/MyDrive/Metal Detection Data/Processed_Data/train"
valid_data_path = "/content/drive/MyDrive/Metal Detection Data/Processed_Data/valid"
test_data_path = "/content/drive/MyDrive/Metal Detection Data/Processed_Data/test"

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_data = datasets.ImageFolder(train_data_path, transform=transform)
valid_data = datasets.ImageFolder(valid_data_path, transform=transform)
test_data = datasets.ImageFolder(test_data_path, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

class_names = train_data.classes
class_dict = train_data.class_to_idx

In [5]:
class TinyVGG_v2(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 64 * 64, out_features=output_shape),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x



device = 'cuda' if torch.cuda.is_available() else 'cpu'


model = TinyVGG_v2(
    input_shape=3,
    hidden_units=64,
    output_shape=len(class_names)
).to(device)

print(f"Fixed model parameters: {sum(p.numel() for p in model.parameters())}")

Fixed model parameters: 1685446


In [6]:
class GeneticAlgorithm:
    """
    Genetic Algorithm implementation for hyperparameter optimization
    """
    def __init__(self,
                 objective_function,
                 dimension,
                 population_size=20,
                 max_generations=50,
                 bounds=(0, 1),
                 crossover_rate=0.8,
                 mutation_rate=0.1,
                 tournament_size=3,
                 elite_size=2,
                 random_seed=None):

        self.objective_function = objective_function
        self.dimension = dimension
        self.population_size = population_size
        self.max_generations = max_generations
        self.bounds = bounds
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.tournament_size = tournament_size
        self.elite_size = elite_size

        if random_seed is not None:
            np.random.seed(random_seed)
            random.seed(random_seed)
            torch.manual_seed(random_seed)

        # Initialize population
        self.population = np.random.uniform(bounds[0], bounds[1],
                                          (population_size, dimension))
        self.fitness_scores = np.full(population_size, np.inf)

        # Track best solution
        self.best_individual = None
        self.best_fitness = np.inf
        self.convergence_history = []

    def evaluate_population(self):
        """Evaluate fitness for entire population"""
        for i in range(self.population_size):
            fitness = self.objective_function(self.population[i])
            self.fitness_scores[i] = fitness

            if fitness < self.best_fitness:
                self.best_fitness = fitness
                self.best_individual = self.population[i].copy()

    def tournament_selection(self):
        """Tournament selection for parent selection"""
        selected_parents = []

        for _ in range(self.population_size - self.elite_size):
            # Tournament selection
            tournament_indices = np.random.choice(self.population_size,
                                                self.tournament_size, replace=False)
            tournament_fitness = self.fitness_scores[tournament_indices]
            winner_idx = tournament_indices[np.argmin(tournament_fitness)]
            selected_parents.append(self.population[winner_idx].copy())

        return np.array(selected_parents)

    def blend_crossover(self, parent1, parent2, alpha=0.5):
        """BLX-α (Blend Crossover) for real-valued genes"""
        child1 = np.zeros(self.dimension)
        child2 = np.zeros(self.dimension)

        for i in range(self.dimension):
            if np.random.random() < self.crossover_rate:
                # BLX-α crossover
                min_val = min(parent1[i], parent2[i])
                max_val = max(parent1[i], parent2[i])
                diff = max_val - min_val

                lower_bound = max(self.bounds[0], min_val - alpha * diff)
                upper_bound = min(self.bounds[1], max_val + alpha * diff)

                child1[i] = np.random.uniform(lower_bound, upper_bound)
                child2[i] = np.random.uniform(lower_bound, upper_bound)
            else:
                child1[i] = parent1[i]
                child2[i] = parent2[i]

        return child1, child2

    def gaussian_mutation(self, individual, sigma=0.1):
        """Gaussian mutation for real-valued genes"""
        mutated = individual.copy()

        for i in range(self.dimension):
            if np.random.random() < self.mutation_rate:
                # Add Gaussian noise
                mutation_strength = sigma * (self.bounds[1] - self.bounds[0])
                mutated[i] += np.random.normal(0, mutation_strength)

                # Ensure bounds
                mutated[i] = np.clip(mutated[i], self.bounds[0], self.bounds[1])

        return mutated

    def adaptive_parameters(self, generation):
        """Adaptive mutation rate - decreases over generations"""
        initial_mutation = self.mutation_rate
        final_mutation = self.mutation_rate * 0.1

        # Linear decay
        adaptive_mutation = initial_mutation - (initial_mutation - final_mutation) * (generation / self.max_generations)
        return adaptive_mutation

    def create_new_generation(self, generation):
        """Create new generation through selection, crossover, and mutation"""
        new_population = []

        # Elitism - keep best individuals
        elite_indices = np.argsort(self.fitness_scores)[:self.elite_size]
        for idx in elite_indices:
            new_population.append(self.population[idx].copy())

        # Tournament selection
        parents = self.tournament_selection()

        # Crossover and mutation
        adaptive_mutation_rate = self.adaptive_parameters(generation)

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                parent1 = parents[i]
                parent2 = parents[i + 1]

                # Crossover
                child1, child2 = self.blend_crossover(parent1, parent2)

                # Mutation
                self.mutation_rate = adaptive_mutation_rate
                child1 = self.gaussian_mutation(child1)
                child2 = self.gaussian_mutation(child2)

                new_population.extend([child1, child2])
            else:
                # Odd number of parents, mutate the last one
                child = self.gaussian_mutation(parents[i])
                new_population.append(child)

        # Ensure population size
        self.population = np.array(new_population[:self.population_size])

    def optimize(self, verbose=True):
        """Run Genetic Algorithm optimization"""
        if verbose:
            print("Starting Genetic Algorithm optimization...")
            print(f"Population size: {self.population_size}")
            print(f"Dimensions: {self.dimension}")
            print(f"Max generations: {self.max_generations}")
            print(f"Crossover rate: {self.crossover_rate}")
            print(f"Mutation rate: {self.mutation_rate}")
            print("-" * 50)

        # Initial evaluation
        self.evaluate_population()
        self.convergence_history.append(self.best_fitness)

        if verbose:
            print(f"Generation   0: Best fitness = {self.best_fitness:.6f}")

        # Evolution loop
        for generation in range(1, self.max_generations + 1):
            # Create new generation
            self.create_new_generation(generation - 1)

            # Evaluate new population
            self.evaluate_population()
            self.convergence_history.append(self.best_fitness)

            if verbose and generation % 5 == 0:
                avg_fitness = np.mean(self.fitness_scores)
                print(f"Generation {generation:3d}: Best = {self.best_fitness:.6f}, "
                      f"Avg = {avg_fitness:.6f}, "
                      f"Std = {np.std(self.fitness_scores):.6f}")

        if verbose:
            print("-" * 50)
            print(f"Evolution completed!")
            print(f"Best fitness: {self.best_fitness:.6f}")

        return {
            'best_individual': self.best_individual,
            'best_fitness': self.best_fitness,
            'convergence_history': self.convergence_history,
            'final_population': self.population,
            'final_fitness_scores': self.fitness_scores
        }

In [7]:
class GA_CNN_Trainer:
    """CNN Trainer with Genetic Algorithm"""

    def __init__(self,
                 train_loader,
                 valid_loader,
                 test_loader,
                 num_classes,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):

        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.test_loader = test_loader
        self.num_classes = num_classes
        self.device = device

        # Parameter bounds for GA optimization
        self.param_bounds = {
            'learning_rate': (1e-5, 1e-1),    # log scale
            'hidden_units': (8, 128),         # linear scale
            'dropout_rate': (0.0, 0.7),       # linear scale
            'weight_decay': (1e-6, 1e-2),     # log scale
            'batch_norm': (0, 1)              # binary choice (0 or 1)
        }

    def vector_to_params(self, vector):
        """Convert GA vector to CNN parameters"""
        params = {}

        # Learning rate (log scale)
        params['learning_rate'] = 10 ** (vector[0] * (np.log10(self.param_bounds['learning_rate'][1]) -
                                                      np.log10(self.param_bounds['learning_rate'][0])) +
                                        np.log10(self.param_bounds['learning_rate'][0]))

        # Hidden units (linear scale)
        params['hidden_units'] = int(vector[1] * (self.param_bounds['hidden_units'][1] -
                                                 self.param_bounds['hidden_units'][0]) +
                                    self.param_bounds['hidden_units'][0])

        # Dropout rate (linear scale)
        params['dropout_rate'] = vector[2] * (self.param_bounds['dropout_rate'][1] -
                                             self.param_bounds['dropout_rate'][0]) + \
                                self.param_bounds['dropout_rate'][0]

        # Weight decay (log scale)
        params['weight_decay'] = 10 ** (vector[3] * (np.log10(self.param_bounds['weight_decay'][1]) -
                                                     np.log10(self.param_bounds['weight_decay'][0])) +
                                       np.log10(self.param_bounds['weight_decay'][0]))

        # Batch normalization (binary choice)
        params['use_batch_norm'] = vector[4] > 0.5

        return params

    def create_model_with_options(self, hidden_units, dropout_rate, use_batch_norm):
        """Create TinyVGG model with optional batch normalization and dropout"""

        if use_batch_norm:
            # Model with batch normalization
            conv_block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(hidden_units),
                nn.ReLU(),
                nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(hidden_units),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
            )
            conv_block_2 = nn.Sequential(
                nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
                nn.BatchNorm2d(hidden_units),
                nn.ReLU(),
                nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
                nn.BatchNorm2d(hidden_units),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
            )
        else:
            # Standard model without batch normalization
            conv_block_1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
            )
            conv_block_2 = nn.Sequential(
                nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2, stride=2),
            )

        # Create model
        model = nn.Module()
        model.conv_block_1 = conv_block_1
        model.conv_block_2 = conv_block_2
        model.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate),
            nn.Linear(in_features=hidden_units * 64 * 64, out_features=self.num_classes),
        )

        # Define forward method
        def forward(x):
            x = model.conv_block_1(x)
            x = model.conv_block_2(x)
            x = model.classifier(x)
            return x

        model.forward = forward
        return model.to(self.device)

    def train_and_evaluate(self, params, epochs=8):
        """Train CNN with given parameters and return validation loss"""
        # Create model with options
        model = self.create_model_with_options(
            params['hidden_units'],
            params['dropout_rate'],
            params['use_batch_norm']
        )

        # Setup training
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                   lr=params['learning_rate'],
                                   weight_decay=params['weight_decay'])

        # Training loop
        model.train()
        for epoch in range(epochs):
            epoch_loss = 0
            batch_count = 0

            for batch_idx, (data, target) in enumerate(self.train_loader):
                if batch_idx > 30:  # Limit training for faster GA iterations
                    break

                data, target = data.to(self.device), target.to(self.device)
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                batch_count += 1

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for data, target in self.valid_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = model(data)
                val_loss += criterion(output, target).item()
                pred = output.argmax(dim=1)
                val_correct += pred.eq(target).sum().item()
                val_total += target.size(0)

        val_accuracy = val_correct / val_total
        avg_val_loss = val_loss / len(self.valid_loader)

        # Return fitness (what GA minimizes) with penalty for low accuracy
        fitness = avg_val_loss + (1 - val_accuracy) * 0.4
        return fitness, model, val_accuracy

    def objective_function(self, vector):
        """Objective function for GA"""
        # Ensure vector is in bounds [0, 1]
        vector = np.clip(vector, 0, 1)
        params = self.vector_to_params(vector)
        fitness, _, _ = self.train_and_evaluate(params, epochs=5)  # Short training for GA
        return fitness

    def run_optimization(self,
                        population_size=20,
                        max_generations=30,
                        crossover_rate=0.8,
                        mutation_rate=0.15,
                        tournament_size=3,
                        elite_size=2,
                        verbose=True):
        """Run GA optimization"""

        ga = GeneticAlgorithm(
            objective_function=self.objective_function,
            dimension=5,  # 5 parameters to optimize (including batch_norm)
            population_size=population_size,
            max_generations=max_generations,
            bounds=(0, 1),  # Normalized bounds
            crossover_rate=crossover_rate,
            mutation_rate=mutation_rate,
            tournament_size=tournament_size,
            elite_size=elite_size,
            random_seed=42
        )

        result = ga.optimize(verbose=verbose)

        # Get best parameters
        best_params = self.vector_to_params(result['best_individual'])

        # Train final model with best parameters
        print("\nTraining final model with best parameters...")
        print(f"Best parameters: {best_params}")

        final_fitness, final_model, final_accuracy = self.train_and_evaluate(
            best_params, epochs=25
        )

        return {
            'best_params': best_params,
            'final_model': final_model,
            'final_accuracy': final_accuracy,
            'convergence_history': result['convergence_history'],
            'final_population': result['final_population'],
            'diversity_stats': {
                'final_std': np.std(result['final_fitness_scores']),
                'final_mean': np.mean(result['final_fitness_scores'])
            }
        }

    def test_model(self, model):
        """Test the final model"""
        model.eval()
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for data, target in self.test_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = model(data)
                pred = output.argmax(dim=1)
                test_correct += pred.eq(target).sum().item()
                test_total += target.size(0)

        test_accuracy = test_correct / test_total
        print(f"Test Accuracy: {test_accuracy:.4f}")
        return test_accuracy


def run_ga_training(train_loader, valid_loader, test_loader, num_classes,
                    population_size=20, max_generations=30, crossover_rate=0.8,
                    mutation_rate=0.15, tournament_size=3, elite_size=2):
    """Main function to run GA training"""

    # Initialize trainer
    trainer = GA_CNN_Trainer(
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        num_classes=num_classes
    )

    # Run optimization
    result = trainer.run_optimization(
        population_size=population_size,
        max_generations=max_generations,
        crossover_rate=crossover_rate,
        mutation_rate=mutation_rate,
        tournament_size=tournament_size,
        elite_size=elite_size,
        verbose=True
    )

    # Test final model
    test_accuracy = trainer.test_model(result['final_model'])

    # Plot convergence and diversity
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(result['convergence_history'], 'g-', linewidth=2, marker='o', markersize=4)
    plt.title('GA Convergence Curve')
    plt.xlabel('Generation')
    plt.ylabel('Best Fitness')
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 3, 2)
    # Plot last 10 generations for detail
    if len(result['convergence_history']) > 10:
        plt.plot(range(len(result['convergence_history'])-10, len(result['convergence_history'])),
                result['convergence_history'][-10:], 'g-', linewidth=2, marker='o', markersize=6)
        plt.title('GA Convergence (Last 10 Generations)')
        plt.xlabel('Generation')
        plt.ylabel('Best Fitness')
        plt.grid(True, alpha=0.3)

    plt.subplot(1, 3, 3)
    # Show parameter distribution in final population
    final_params = [trainer.vector_to_params(ind) for ind in result['final_population']]
    learning_rates = [p['learning_rate'] for p in final_params]
    hidden_units = [p['hidden_units'] for p in final_params]

    plt.scatter(learning_rates, hidden_units, alpha=0.6, c='green')
    plt.xlabel('Learning Rate')
    plt.ylabel('Hidden Units')
    plt.title('Final Population Distribution\n(Learning Rate vs Hidden Units)')
    plt.xscale('log')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return result, test_accuracy



In [ ]:
print("\n" + "="*60)
print("STARTING GENETIC ALGORITHM HYPERPARAMETER OPTIMIZATION")
print("="*60)

# Configuration 1: Standard GA
print("\n1. Running Standard GA Configuration...")
result_standard, test_acc_standard = run_ga_training(
    train_loader, valid_loader, test_loader,
    num_classes=len(class_names),
    population_size=16,
    max_generations=14,
    crossover_rate=0.8,
    mutation_rate=0.15,
    tournament_size=3,
    elite_size=2
)

print(f"\nStandard GA Results:")
print(f"Final test accuracy: {test_acc_standard:.4f}")
print(f"Best parameters: {result_standard['best_params']}")


STARTING GENETIC ALGORITHM HYPERPARAMETER OPTIMIZATION

1. Running Standard GA Configuration...
Starting Genetic Algorithm optimization...
Population size: 16
Dimensions: 5
Max generations: 14
Crossover rate: 0.8
Mutation rate: 0.15
--------------------------------------------------
Generation   0: Best fitness = 0.094095
Generation   5: Best = 0.068130, Avg = 0.369946, Std = 0.288234


In [ ]:
# Configuration 2: High Mutation GA (More Exploration)
print("\n2. Running High Mutation GA Configuration...")
result_high_mutation, test_acc_high_mutation = run_ga_training(
    train_loader, valid_loader, test_loader,
    num_classes=len(class_names),
    population_size=20,
    max_generations=30,
    crossover_rate=0.7,
    mutation_rate=0.25,    # Higher mutation rate
    tournament_size=4,
    elite_size=1
)

print(f"\nHigh Mutation GA Results:")
print(f"Final test accuracy: {test_acc_high_mutation:.4f}")
print(f"Best parameters: {result_high_mutation['best_params']}")

In [ ]:
# Configuration 3: Elitist GA (More Exploitation)
print("\n3. Running Elitist GA Configuration...")
result_elitist, test_acc_elitist = run_ga_training(
    train_loader, valid_loader, test_loader,
    num_classes=len(class_names),
    population_size=15,
    max_generations=35,
    crossover_rate=0.9,
    mutation_rate=0.08,    # Lower mutation rate
    tournament_size=5,     # Larger tournament
    elite_size=4          # More elites
)

print(f"\nElitist GA Results:")
print(f"Final test accuracy: {test_acc_elitist:.4f}")
print(f"Best parameters: {result_elitist['best_params']}")

In [ ]:
# Configuration 4: Diverse GA (Balanced)
print("\n4. Running Diverse GA Configuration...")
result_diverse, test_acc_diverse = run_ga_training(
    train_loader, valid_loader, test_loader,
    num_classes=len(class_names),
    population_size=25,    # Larger population
    max_generations=20,
    crossover_rate=0.85,
    mutation_rate=0.12,
    tournament_size=2,     # Smaller tournament for diversity
    elite_size=3
)

print(f"\nDiverse GA Results:")
print(f"Final test accuracy: {test_acc_diverse:.4f}")
print(f"Best parameters: {result_diverse['best_params']}")